### Using correctionlib for btag

In [1]:
import uproot3
import numpy as np
import os
from coffea import hist, lookup_tools
# from coffea.lookup_tools import extractor, dense_lookup
from coffea import util 
from coffea.lookup_tools import dense_lookup
from coffea.util import save, load 
from coffea.btag_tools import BTagScaleFactor
import awkward as ak
import json 
from coffea.lookup_tools import extractor

import correctionlib
import hist
import csv
import pandas as pd


from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools.correctionlib_wrapper import correctionlib_wrapper
from coffea.lookup_tools.dense_lookup import dense_lookup


## Convert the CSV file
from Ac Malik Williams

Convert the CSV file for new format

In [99]:
def FormatCSV(filename):
    """
    filename ---> String (bTag SF .csv file)
    """
    ############### Ensures that file's workingpoint is written as an integer ##################
    
    workingpoints = {
        'L':0,
        'M':1,
        'T':2,
        'R':3
    }
    
    SF_stuff = [line.split(',') for line in open(filename)] # "Stuff" in the file    
    #print(SF_stuff)
    SF_stuff_array = np.array(SF_stuff) # Convert the lines into numpy array (for slicing)
    print(SF_stuff_array)
    #print(SF_stuff_array[2][0])
    #OperatingPoints = SF_stuff_array[2][0] # Select only the first element of each line ("Operating Points")
    OperatingPoints = SF_stuff_array[1:,0] # Select only the first element of each line ("Operating Points")
    
    IntegerPoints = [workingpoints[letter] for letter in OperatingPoints] # Convert the strings to integers
    dataframe = pd.read_csv(filename) # Prep to swap out the "string" column for the new "integer" column
    dataframe.OperatingPoint = IntegerPoints # Complete the swap
    print(filename+' ---> '+filename[:-4]+'_converted.csv') # Optional confirmation
    ConvertedFile = dataframe.to_csv(filename[:-4]+'_converted.csv', index = False) # Output new converted csv file
  
    return ConvertedFile

In [108]:
csvfile = "data/btag_SF/UL/wp_deepJet_106XUL17_v3.csv" #"data/btag_SF/UL/wp_deepJet_106XUL16preVFP_v2.csv"
cnt = 0
for line in open(csvfile):
    print(cnt)
    print(line)
    print(line.split(','))
    stuff = [line.split(',')]
    print(stuff)
    cnt = cnt +1
    if cnt > 5: break
print("=====================================================================")
csvfile2 = "data/btag_SF/UL/wp_deepJet_106XUL16preVFP_v2.csv"
cnt2 = 0
for line in open(csvfile2):
    print(cnt2)
    print(line)
    print(line.split(','))
    cnt2 = cnt2 +1
    if cnt2 > 2: break

0
OperatingPoint,measurementType,sysType,jetFlavor,etaMin,etaMax,ptMin,ptMax,discrMin,discrMax,formula 

['OperatingPoint', 'measurementType', 'sysType', 'jetFlavor', 'etaMin', 'etaMax', 'ptMin', 'ptMax', 'discrMin', 'discrMax', 'formula \n']
[['OperatingPoint', 'measurementType', 'sysType', 'jetFlavor', 'etaMin', 'etaMax', 'ptMin', 'ptMax', 'discrMin', 'discrMax', 'formula \n']]
1
L,incl,central,0,0.0,2.5,20.0,1000.0,0.0,1.0,1.34198+-0.000555031*x+3.20633e-07*x*x+0.888495/x 

['L', 'incl', 'central', '0', '0.0', '2.5', '20.0', '1000.0', '0.0', '1.0', '1.34198+-0.000555031*x+3.20633e-07*x*x+0.888495/x \n']
[['L', 'incl', 'central', '0', '0.0', '2.5', '20.0', '1000.0', '0.0', '1.0', '1.34198+-0.000555031*x+3.20633e-07*x*x+0.888495/x \n']]
2
L,incl,down,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.34198+-0.000555031*x+3.20633e-07*x*x+0.888495/x)*(1-(0.0663918+6.22127e-05*x+-9.14037e-08*x*x)) 

['L', 'incl', 'down', '0', '0.0', '2.5', '20.0', '1000.0', '0.0', '1.0', '(1.34198+-0.000555031*x+3.20633e-

In [ ]:
csvfiles = [
#    'wp_deepCSV_106XUL16postVFP_v3.csv',
#    'wp_deepCSV_106XUL17_v3.csv',
#    'wp_deepJet_106XUL17_v3.csv',
    'wp_deepJet_106XUL16postVFP_v3.csv',
    'wp_deepJet_106XUL18_v2.csv',
#    'wp_deepCSV_106XUL16preVFP_v2.csv',
#    'wp_deepCSV_106XUL18_v2.csv',
    'wp_deepJet_106XUL16preVFP_v2.csv'
]
#for file in csvfiles:
#    csvfile = 'data/btag_SF/UL/'+file
#    print(csvfile)
#    FormatCSV(csvfile)
#csvfile = "data/btag_SF/UL/wp_deepJet_106XUL16postVFP_v3.csv"
#csvfile = "data/btag_SF/UL/wp_deepJet_106XUL17_v3.csv"
csvfile = "data/btag_SF/UL/wp_deepJet_106XUL16preVFP_v2.csv"

convfile = FormatCSV(csvfile)

## BTagCorrector definition from correction.py


In [3]:
class BTagCorrectorPreVFP(object):

    def __init__(self, tagger, year, workingpoint):
        self._year = year 
        files = {
           'deepflav': {
               '2016': 'wp_deepJet_106XUL16preVFP_v2.csv',
               '2017': 'wp_deepJet_106XUL17_v3_converted.csv',
               '2018': 'wp_deepJet_106XUL18_v2.csv'
           },   
           'deepcsv': {
                '2016': 'wp_deepCSV_106XUL16preVFP_v2.csv',
                '2017': 'wp_deepCSV_106XUL17_v3.csv',
                '2018': 'wp_deepCSV_106XUL18_v2.csv'
           }    
        }    

        common = load('data/common.coffea')
        self._wp = common['btagWPs'][tagger][year][workingpoint]
        filename = 'data/btag_SF/UL/'+files[tagger][year]
        self.sf = BTagScaleFactor(filename, workingpoint)
        files = {
            '2016': 'btag2016.merged',
            '2017': 'btag2017.merged',
            '2018': 'btag2018.merged',
        }    
        filename = 'hists/'+files[year]
        btag = load(filename)
        bpass = btag[tagger].integrate('dataset').integrate('wp',workingpoint).integrate('btag', 'pass').values()[()]
        ball = btag[tagger].integrate('dataset').integrate('wp',workingpoint).integrate('btag').values()[()]
        nom = bpass / np.maximum(ball, 1.)
        #self.eff = dense_lookup.dense_lookup(nom, [ax.edges() for ax in btag[tagger].axes()[3:]])
        self.eff = dense_lookup(nom, [ax.edges() for ax in btag[tagger].axes()[3:]])
        
    def btag_weight(self, events, pt, eta, flavor, tag):
        print('events:', events)
        print('pt:', pt)
        print('eta:', eta)
        print('flavor:', flavor)
        print('tag:', tag)
        abseta = abs(eta)
        tightJet = events
        #https://twiki.cern.ch/twiki/bin/viewauth/CMS/BTagSFMethods#1b_Event_reweighting_using_scale
        def zerotag(eff):
            return ak.prod(1.0 - eff[tightJet.isdflvM], axis=-1)
             
        def onetag(eff):
            output = ak.prod(eff[tightJet.isdflvM], axis=-1) * ak.prod((1.0 - eff[np.invert(tightJet.isdflvM)]), axis=-1)
            return(output)
        
        eff = self.eff(flavor, pt, abseta)
        sf_nom = self.sf.eval('central', flavor, abseta, pt)
        sf_up = self.sf.eval('up', flavor, abseta, pt)
        sf_down = self.sf.eval('down', flavor, abseta, pt)

        eff_data_nom  = np.minimum(1., sf_nom*eff)
        eff_data_up   = np.minimum(1., sf_up*eff)
        eff_data_down = np.minimum(1., sf_down*eff)


        if '-1' in tag: 
            nom = (1 - zerotag(eff_data_nom)) / (1 - zerotag(eff))
            up = (1 - zerotag(eff_data_up)) / (1 - zerotag(eff))
            down = (1 - zerotag(eff_data_down)) / (1 - zerotag(eff))
        elif '2' in tag:
            nom = (1- zerotag(eff_data_nom) - onetag(eff_data_nom))/(1- zerotag(eff) - onetag(eff))
            up =(1- zerotag(eff_data_up) - onetag(eff_data_up))/(1- zerotag(eff) - onetag(eff))
            down =(1- zerotag(eff_data_down) - onetag(eff_data_down))/(1- zerotag(eff) - onetag(eff))
        elif '+1' in tag:
            nom = onetag(eff_data_nom)/onetag(eff)
            up= onetag(eff_data_up)/onetag(eff)
            down = onetag(eff_data_down)/onetag(eff)
        else:
            nom = zerotag(eff_data_nom)/zerotag(eff)
            up = zerotag(eff_data_up)/zerotag(eff)
            down = zerotag(eff_data_down)/zerotag(eff)

        return np.nan_to_num(nom), np.nan_to_num(up), np.nan_to_num(down)


In [8]:

onefiles = [
    #"./Mphi-1995_Mchi-1000_2018_1.root",
    "root://xrootd-cms.infn.it//store/user/empfeffe/customNano/DYJetsToLL_0J_TuneCP5_13TeV-amcatnloFXFX-pythia8/KIT_CustomNanoV9_MC_2018/211220_132108/0000/MC_2018_NanoAODv9_189.root",
]
fnameMCs = [
    #"root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_1.root",
    #"root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_2.root",
    "root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_3.root"
]

events = NanoEventsFactory.from_root(
    #onefiles[0],
    "root://cmsxrootd-site.fnal.gov//store/user/empfeffe/customNano/DYJetsToLL_0J_TuneCP5_13TeV-amcatnloFXFX-pythia8/KIT_CustomNanoV9_MC_2018/211220_132108/0000/MC_2018_NanoAODv9_189.root",
    #"root://cmsxrootd-site.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root",
    entry_stop=100_000,
    schemaclass=NanoAODSchema,
).events()

OSError: XRootD error: [ERROR] Operation expired
in file root://cmsxrootd-site.fnal.gov//store/user/empfeffe/customNano/DYJetsToLL_0J_TuneCP5_13TeV-amcatnloFXFX-pythia8/KIT_CustomNanoV9_MC_2018/211220_132108/0000/MC_2018_NanoAODv9_189.root

In [ ]:
events.Jet.fields

In [37]:
deepflavWPs = { 
    '2016':{ 
            'loose' :  0.0508,
            'medium':  0.2598,
            'tight' :  0.6502 
    
    },  
    '2017': {
        'loose' : 0.0532,
        'medium': 0.3040,
        'tight' : 0.7476
    },  
    '2018': {
        'loose' : 0.0490,
        'medium': 0.2783,
        'tight' : 0.7100    
    },  
}

jet = events.Jet
jet['isdflvL'] = (jet.btagDeepFlavB > deepflavWPs['2017']['loose'])
jet['isdflvM'] = (jet.btagDeepFlavB > deepflavWPs['2017']['medium'])
btag_weight = BTagCorrectorPreVFP('deepflav','2017','loose' ).btag_weight

btag={}
btagUp={}
btagDown={}
btag['sr'], btagUp['sr'], btagDown['sr'] = btag_weight(jet, jet.pt, jet.eta, jet.hadronFlavour, '0', )

events: [[Jet, Jet, Jet, Jet, Jet], [Jet, Jet, Jet, ... Jet], [Jet, Jet, Jet, Jet, Jet, Jet]]
pt: [[255, 71.4, 25.3, 20, 15.6], [291, 232, ... 18], [92.2, 87.3, 72, 16.2, 15.5, 15.1]]
eta: [[0.398, 3.12, 0.00673, -1.84, 2.94], ... -1.84, -1.55, -2.84, -3.35, 2.68, 3.21]]
flavor: [[0, 0, 0, 0, 0], [0, 0, 0], [5, 0, 0, ... 0, 0, 0, 0, 0, 0, 0], [0, 5, 0, 0, 0, 0]]
tag: 0


In [38]:
print('btag sr:', btag['sr'])
print('btagUp: ', btagUp['sr'])
print('btagDown:', btagDown['sr'])

btag sr: [1, 1, 1.14, 1, 1.18, 1.39, 1.29, 1.14, ... 1.04, 1, 1, 1, 0.95, 1.48, 1.17, 1.14]
btagUp:  [1, 1, 1.01, 1, 1, 0.825, 0.931, 0.997, ... 0.92, 1, 1, 1, 0.939, 0.904, 1.03, 1.03]
btagDown: [1, 1, 1.28, 1, 1.36, 1.95, 1.64, 1.28, ... 1.16, 1, 1, 1, 0.96, 2.19, 1.31, 1.26]


In [3]:
files = {
    'deepflav': {
        '2016': 'wp_deepJet_106XUL16preVFP_v2.csv',
        '2017': 'wp_deepJet_106XUL17_v3_converted.csv',
        '2018': 'wp_deepJet_106XUL18_v2.csv'
    },   
    'deepcsv': {
        '2016': 'wp_deepCSV_106XUL16preVFP_v2.csv',
        '2017': 'wp_deepCSV_106XUL17_v3.csv',
        '2018': 'wp_deepCSV_106XUL18_v2.csv'
    }    
}
tagger = 'deepflav'
year = '2017'
#filename = 'data/btag_SF/UL/'+files[tagger][year]
filename = 'data/DeepJet_2016LegacySF_V1.csv'
workingpoint = 'loose'

sf = BTagScaleFactor(filename, workingpoint)
#sf_nom = sf.eval('central', flavor, abseta, pt)
print(sf)

1 dimensional histogram with axes:
	1: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71.
 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. 88. 89.
 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]



In [2]:
import correctionlib
import hist
import awkward as ak
import numpy as np

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools.correctionlib_wrapper import correctionlib_wrapper
from coffea.lookup_tools.dense_lookup import dense_lookup

In [3]:
#1
events = NanoEventsFactory.from_root(
    #"root://cmsxrootd-site.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root",
    "root://xrootd-cms.infn.it//store/user/nshadski/customNano/QCD_Pt_30to50_TuneCP5_13TeV_pythia8/KIT_CustomNanoV9_MC_2018/211221_114254/0000/MC_2018_NanoAODv9_120.root",
    entry_stop=100_000,
    schemaclass=NanoAODSchema,
).events()

In [19]:
phasespace_cuts = (
    (abs(events.Jet.eta) < 2.5)
    & (events.Jet.pt > 40.)
)
jets = ak.flatten(events.Jet[phasespace_cuts])

efficiencyinfo = (
    hist.Hist.new
    .Reg(20, 40, 300, name="pt")
    .Reg(4, 0, 2.5, name="abseta")
    .IntCat([0, 4, 5], name="flavor")
    .Bool(name="passWP")
    .Double()
    .fill(
        pt=jets.pt,
        abseta=abs(jets.eta),
        flavor=jets.hadronFlavour,
        passWP=jets.btagDeepFlavB > 0.2783, # UL 2018 medium WP
    )
)

In [16]:
phasespace_cuts = (
    (abs(events.Jet.eta) < 2.5)
    & (events.Jet.pt > 40.)
)
jets = ak.flatten(events.Jet[phasespace_cuts])

deepflavWPs = {'loose': 0.0490, 'medium': 0.2783, 'tight': 0.7100}
deepcsvWPs  = {'loose': 0.1241, 'medium': 0.4184, 'tight': 0.7527}
btagWPs = {'deepflav': deepflavWPs, 'deepcsv': deepcsvWPs}
name = {}
name['deepflav']= 'btagDeepFlavB'
name['deepcsv']= 'btagDeepB'
test = {}
for wp in ['loose', 'medium', 'tight']:
    for tagger in ['deepflav', 'deepcsv']:
        #passbtag = jets[name[tagger]] > 0.2783
        test[tagger] = (hist.Hist.new
        .Reg(20, 20, 1000, name='pt')
        .Reg(4,0,2.5, name='abseta')
        .IntCat([0, 4, 5, 6], name='flavor')
        .Bool(name='passWP')
        .Double()
        .fill(
            pt=jets.pt,
            abseta=abs(jets.eta),
            flavor=jets.hadronFlavour,
            passWP=jets[name[tagger]] > btagWPs[tagger][wp]
        ))

KeyError: 'deepflav'

In [4]:
#2
phasespace_cuts = (
    (abs(events.Jet.eta) < 2.5)
    & (events.Jet.pt > 40.)
)
jets = ak.flatten(events.Jet[phasespace_cuts])

deepflavWPs = {'loose': 0.0490, 'medium': 0.2783, 'tight': 0.7100}
deepcsvWPs  = {'loose': 0.1241, 'medium': 0.4184, 'tight': 0.7527}
btagWPs = {'deepflav': deepflavWPs, 'deepcsv': deepcsvWPs}
name = {}
name['deepflav']= 'btagDeepFlavB'
name['deepcsv']= 'btagDeepB'
btag_deepflav = {}
btag_deepcsv = {}
for wp in ['loose', 'medium', 'tight']:
    btag_deepflav[wp] = (hist.Hist.new
                           .Reg(20,20, 1000, name='pt')
                           .Reg(4, 0, 2.5, name='abseta')
                           .IntCat([0,4,5,6], name='flavor')
                           .Bool(name='passWP')
                           .Double()
                           .fill(
                               pt=jets.pt,
                               abseta=abs(jets.eta),
                               flavor=jets.hadronFlavour,
                               passWP=jets[name['deepflav']]>btagWPs['deepflav'][wp]))
for wp in ['loose', 'medium', 'tight']:
    btag_deepcsv[wp] = (hist.Hist.new
                           .Reg(20,20, 1000, name='pt')
                           .Reg(4, 0, 2.5, name='abseta')
                           .IntCat([0,4,5,6], name='flavor')
                           .Bool(name='passWP')
                           .Double()
                           .fill(
                               pt=jets.pt,
                               abseta=abs(jets.eta),
                               flavor=jets.hadronFlavour,
                               passWP=jets[name['deepcsv']]>btagWPs['deepcsv'][wp]))

In [5]:
#2-1
for wp in ['loose', 'medium', 'tight']: 
    print(btag_deepflav[wp][{'passWP':True}])
    print(btag_deepcsv[wp][{'passWP':True}])

Hist(
  Regular(20, 20, 1000, name='pt', label='pt'),
  Regular(4, 0, 2.5, name='abseta', label='abseta'),
  IntCategory([0, 4, 5, 6], name='flavor', label='flavor'),
  storage=Double()) # Sum: 163.0
Hist(
  Regular(20, 20, 1000, name='pt', label='pt'),
  Regular(4, 0, 2.5, name='abseta', label='abseta'),
  IntCategory([0, 4, 5, 6], name='flavor', label='flavor'),
  storage=Double()) # Sum: 143.0
Hist(
  Regular(20, 20, 1000, name='pt', label='pt'),
  Regular(4, 0, 2.5, name='abseta', label='abseta'),
  IntCategory([0, 4, 5, 6], name='flavor', label='flavor'),
  storage=Double()) # Sum: 40.0
Hist(
  Regular(20, 20, 1000, name='pt', label='pt'),
  Regular(4, 0, 2.5, name='abseta', label='abseta'),
  IntCategory([0, 4, 5, 6], name='flavor', label='flavor'),
  storage=Double()) # Sum: 37.0
Hist(
  Regular(20, 20, 1000, name='pt', label='pt'),
  Regular(4, 0, 2.5, name='abseta', label='abseta'),
  IntCategory([0, 4, 5, 6], name='flavor', label='flavor'),
  storage=Double()) # Sum: 22.0
His

In [20]:
eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
# note this seems to turn 0,4,5 into 0,1,2
efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
#print(eff.values())
efflookup

3 dimensional histogram with axes:
	1: [ 40.  53.  66.  79.  92. 105. 118. 131. 144. 157. 170. 183. 196. 209.
 222. 235. 248. 261. 274. 287. 300.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3.]

In [6]:
#3
eff_flav = {}
eff_csv = {}
efflookup_flav = {}
efflookup_csv = {}
for wp in ['loose', 'medium', 'tight']:
    eff_flav[wp] = btag_deepflav[wp][{'passWP':True}] / btag_deepflav[wp][{'passWP':sum}]
    eff_csv[wp]  = btag_deepcsv[wp][{'passWP':True}] / btag_deepcsv[wp][{'passWP':sum}]
    efflookup_flav[wp] = dense_lookup(eff_flav[wp].values(), [ax.edges for ax in eff_flav[wp].axes])
    efflookup_csv[wp]  = dense_lookup(eff_csv[wp].values(),  [ax.edges for ax in eff_csv[wp].axes])

In [7]:
print(efflookup_flav['loose'](42,0.2, np.array([0,1,2])))
print(efflookup_csv)

[0.06614786 0.65217391 0.66666667]
{'loose': 3 dimensional histogram with axes:
	1: [  20.   69.  118.  167.  216.  265.  314.  363.  412.  461.  510.  559.
  608.  657.  706.  755.  804.  853.  902.  951. 1000.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3. 4.]
, 'medium': 3 dimensional histogram with axes:
	1: [  20.   69.  118.  167.  216.  265.  314.  363.  412.  461.  510.  559.
  608.  657.  706.  755.  804.  853.  902.  951. 1000.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3. 4.]
, 'tight': 3 dimensional histogram with axes:
	1: [  20.   69.  118.  167.  216.  265.  314.  363.  412.  461.  510.  559.
  608.  657.  706.  755.  804.  853.  902.  951. 1000.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3. 4.]
}


In [18]:
efflookup(42, 0.2, np.array([0, 1, 2]))


array([0.01634657, 0.17330046, 0.83132001])

In [13]:
#4
cset = correctionlib.CorrectionSet.from_file("/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/BTV/2018_UL/btagging.json.gz")
#whatis = correctionlib.CorrectionSet.from_file("/cms/scratch/sdogra/mtop/rishabh/decaf/analysis/data/btag_SF/UL/wp_deepJet_106XUL17_v3.csv")
print(cset)
print([c for c in cset])

['deepCSV_comb', 'deepCSV_incl', 'deepCSV_mujets', 'deepCSV_shape', 'deepJet_comb', 'deepJet_incl', 'deepJet_mujets', 'deepJet_shape']


In [46]:
def lighttagSF(j, syst="central"):
    # until correctionlib handles jagged data natively we have to flatten and unflatten
    j, nj = ak.flatten(j), ak.num(j)
    sf = cset["deepJet_incl"].evaluate(syst, "M", np.array(j.hadronFlavour), np.array(abs(j.eta)), np.array(j.pt))
    return ak.unflatten(sf, nj)


def btagSF(j, syst="central"):
    # until correctionlib handles jagged data natively we have to flatten and unflatten
    j, nj = ak.flatten(j), ak.num(j)
    sf = cset["deepJet_comb"].evaluate(syst, "M", np.array(j.hadronFlavour), np.array(abs(j.eta)), np.array(j.pt))
    return ak.unflatten(sf, nj)

In [11]:
#5
def btagSF_deepflav(j, syst):
    j, nj = ak.flatten(j), ak.num(j)
    sf = cset["deepJet_incl"].evaluate(syst, "M", np.array(j.hadronFlavour), np.array(abs(j.eta)), np.array(j.pt))
    return ak.unflatten(sf, nj)

def btagSF_deepdcsv(j, syst):
    j, nj = ak.flatten(j), ak.num(j)
    sf = cset["deepCSV_incl"].evaluate(syst, "M", np.array(j.hadronFlavour), np.array(abs(j.eta)), np.array(j.pt))
    return ak.unflatten(sf, nj)

In [29]:
lightJets = events.Jet[phasespace_cuts & (events.Jet.hadronFlavour == 0)]
bcJets = events.Jet[phasespace_cuts & (events.Jet.hadronFlavour > 0)]

lightEff = efflookup(lightJets.pt, abs(lightJets.eta), lightJets.hadronFlavour)
bcEff = efflookup(bcJets.pt, abs(bcJets.eta), bcJets.hadronFlavour)

In [14]:
lightJets = events.Jet[phasespace_cuts & (events.Jet.hadronFlavour == 0)]
bcJets = events.Jet[phasespace_cuts & (events.Jet.hadronFlavour > 0)]

btagSF_deepflav(lightJets, 'central')

<Array [[1.38], [], [], ... [], [], [1.38]] type='3000 * var * float64'>

In [ ]:
sf_nom={}
sf_nom = efflookup

In [ ]:
get_btag_weight = {
    'loose' : {
        'nom': n,
        'up' : u,
        'down': d
    },
    'medium': b,
    'tight' : c
}

In [32]:
print(lightJets.hadronFlavour)

[[0, 0, 0], [0, 0], [0], [0, 0, 0], [0, ... 0, 0, 0, 0], [0, 0, 0, 0], [], [0, 0, 0]]


In [33]:
def combine(eff, sf, passbtag):
    # tagged SF = SF*eff / eff = SF
    tagged_sf = ak.prod(sf[passbtag], axis=-1)
    # untagged SF = (1 - SF*eff) / (1 - eff)
    untagged_sf = ak.prod(((1 - sf*eff) / (1 - eff))[~passbtag], axis=-1)
    return tagged_sf * untagged_sf

In [37]:
lightweight = combine(
    lightEff,
    lighttagSF(lightJets),
    lightJets.btagDeepB > 0.2783
)
print(lightweight)
bcweight = combine(
    bcEff,
    btagSF(bcJets),
    bcJets.btagDeepB > 0.2783,
)
print(bcweight)
eventweight = lightweight * bcweight
print(eventweight)

[0.986, 1.36, 0.995, 1.36, 1.37, 0.993, 1, ... 0.994, 1, 0.97, 0.982, 1, 0.982]
[1, 1.23, 1.16, 0.935, 0.962, 0.874, 0.913, ... 0.967, 1.13, 0.968, 1, 1.13, 1]
[0.986, 1.67, 1.15, 1.27, 1.32, 0.867, ... 0.961, 1.13, 0.939, 0.982, 1.13, 0.982]


In [17]:
import csv
with open(filename, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

OperatingPoint,measurementType,sysType,jetFlavor,etaMin,etaMax,ptMin,ptMax,discrMin,discrMax,formula, 
L,incl,central,0,0.0,2.5,20.0,1000.0,0.0,1.0,1.46193+-0.000605595*x+3.30224e-07*x*x+-0.367873/x, 
L,incl,down,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.46193+-0.000605595*x+3.30224e-07*x*x+-0.367873/x)*(1-(0.083685+-0.000104849*x+9.12824e-08*x*x)), 
L,incl,down_correlated,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.46193+-0.000605595*x+3.30224e-07*x*x+-0.367873/x)*(1-(0.050987+7.03101e-05*x+-1.09927e-07*x*x)), 
L,incl,down_uncorrelated,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.46193+-0.000605595*x+3.30224e-07*x*x+-0.367873/x)*(1-(0.0381757)), 
L,incl,up,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.46193+-0.000605595*x+3.30224e-07*x*x+-0.367873/x)*(1+(0.083685+-0.000104849*x+9.12824e-08*x*x)), 
L,incl,up_correlated,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.46193+-0.000605595*x+3.30224e-07*x*x+-0.367873/x)*(1+(0.050987+7.03101e-05*x+-1.09927e-07*x*x)), 
L,incl,up_uncorrelated,0,0.0,2.5,20.0,1000.0,0.0,1.0,(1.46193+-0.000605595*x+3.3022

M,comb,up_topmass,4,0.0,2.5,140.0,200.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+0.00084632739890366793, 
M,comb,up_topmass,4,0.0,2.5,200.0,300.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+0.0010412500705569983, 
M,comb,up_topmass,4,0.0,2.5,300.0,600.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+0.0014057601802051067, 
M,comb,up_topmass,4,0.0,2.5,600.0,1000.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+-0.00013494273298420012, 
M,comb,up_topmass,5,0.0,2.5,20.0,30.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+0.00016602622054051608, 
M,comb,up_topmass,5,0.0,2.5,30.0,50.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+0.0004543299728538841, 
M,comb,up_topmass,5,0.0,2.5,50.0,70.0,0.0,1.0,0.764066+0.00814871*log(x+19)*log(x+18)*(3-0.399988*log(x+18))+0.00043463465408422053, 
M,comb,up_topmass,5,0.0,2.5,70.0,100.0,0.0,1.0,0.764066

In [20]:
import pandas as pd
df = pd.read_csv(filename, skipinitialspace=True)
print(df)

     OperatingPoint measurementType            sysType  jetFlavor  etaMin  \
0                 L            incl            central          0     0.0   
1                 L            incl               down          0     0.0   
2                 L            incl    down_correlated          0     0.0   
3                 L            incl  down_uncorrelated          0     0.0   
4                 L            incl                 up          0     0.0   
...             ...             ...                ...        ...     ...   
2188              T          mujets           up_type3          5     0.0   
2189              T          mujets           up_type3          5     0.0   
2190              T          mujets           up_type3          5     0.0   
2191              T          mujets           up_type3          5     0.0   
2192              T          mujets           up_type3          5     0.0   

      etaMax  ptMin   ptMax  discrMin  discrMax  \
0        2.5   20.0  100